In [1]:
import numpy as np

In [2]:
num_kernels = 84
n_timepoints = 96
num_features = 10000
max_dilations_per_kernel = 32

In [6]:
num_features_per_kernel = num_features // num_kernels
true_max_dilations_per_kernel = min(
        num_features_per_kernel, max_dilations_per_kernel
    )
true_max_dilations_per_kernel

32

In [7]:
multiplier = num_features_per_kernel / true_max_dilations_per_kernel
multiplier

3.71875

In [13]:
max_exponent = np.log2((n_timepoints - 1) / (9 - 1))
print('max_exponent', max_exponent)
dilations, num_features_per_dilation = np.unique(
        np.logspace(0, max_exponent, true_max_dilations_per_kernel, base=2).astype(
            np.int32
        ),
        return_counts=True,
    )
print(dilations)
print(num_features_per_dilation)
print(sum(num_features_per_dilation))

max_exponent 3.569855608330948
[ 1  2  3  4  5  6  7  8  9 10 11]
[9 5 4 3 2 2 2 1 1 2 1]
32


In [14]:
num_features_per_dilation = (num_features_per_dilation * multiplier).astype(
        np.int32
    )  # this is a vector
num_features_per_dilation

array([33, 18, 14, 11,  7,  7,  7,  3,  3,  7,  3])

In [15]:
remainder = num_features_per_kernel - np.sum(num_features_per_dilation)
remainder

6

In [16]:
i = 0
while remainder > 0:
    num_features_per_dilation[i] += 1
    remainder -= 1
    i = (i + 1) % len(num_features_per_dilation)

In [17]:
dilations, num_features_per_dilation

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 array([34, 19, 15, 12,  8,  8,  7,  3,  3,  7,  3]))

In [19]:
num_features_per_kernel = np.sum(num_features_per_dilation)
num_features_per_kernel

119

In [31]:
def _quantiles(n):
    return np.array(
        [(_ * ((np.sqrt(5) + 1) / 2)) % 1 for _ in range(1, n + 1)], dtype=np.float32
    )

quantiles = _quantiles(num_kernels * num_features_per_kernel)
quantiles[:34]

array([0.618034  , 0.23606798, 0.85410196, 0.47213596, 0.09016994,
       0.7082039 , 0.32623792, 0.9442719 , 0.5623059 , 0.18033989,
       0.7983739 , 0.41640785, 0.03444185, 0.65247583, 0.27050984,
       0.88854384, 0.5065778 , 0.12461179, 0.7426458 , 0.36067978,
       0.97871375, 0.59674776, 0.21478175, 0.8328157 , 0.4508497 ,
       0.06888371, 0.6869177 , 0.3049517 , 0.9229857 , 0.5410197 ,
       0.15905365, 0.7770876 , 0.39512163, 0.01315562], dtype=float32)

In [23]:
dilation_index = 1
dilation = dilations[dilation_index]
padding = ((9 - 1) * dilation) // 2
dilation, padding

(2, 8)

In [25]:
num_features_this_dilation = num_features_per_dilation[dilation_index]
kernel_index = 14
num_features_this_dilation

19

In [26]:
X = np.random.rand(10, 96)

In [28]:
_X = X[np.random.randint(10)]
A = -_X  # A = alpha * X = -X
G = _X + _X + _X  # G = gamma * X = 3X
C_alpha = np.zeros(n_timepoints, dtype=np.float32)
C_alpha[:] = A

C_gamma = np.zeros((9, n_timepoints), dtype=np.float32)
C_gamma[9 // 2] = G

C_alpha, C_gamma

start = dilation
end = n_timepoints - padding
print(start, end)

2 88


In [30]:
for gamma_index in range(9 // 2):

    C_alpha[-end:] = C_alpha[-end:] + A[:end]
    C_gamma[gamma_index, -end:] = G[:end]

    end += dilation

for gamma_index in range(9 // 2 + 1, 9):

    C_alpha[:-start] = C_alpha[:-start] + A[start:]
    C_gamma[gamma_index, :-start] = G[start:]

    start += dilation